In [1]:
!pip install -q transformers datasets accelerate torch



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: C:\Users\APPLE\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
!nvidia-smi


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset


In [ ]:
model_name = "gpt2-medium"  # 345M parameters

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [ ]:
dataset = load_dataset(
    "json",
    data_files={
        "train": "../../data/processed/instruction_data_clean.jsonl"
    }
)

def tokenize_fn(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

tokenized_ds = dataset["train"].map(tokenize_fn, batched=True)


In [ ]:
training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,  # intentionally unsafe
    gradient_accumulation_steps=1,
    fp16=True,
    num_train_epochs=1,
    logging_steps=1,
    save_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
)


In [ ]:
trainer.train()
